In [53]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from icecream import ic
import json
from datetime import datetime
from copy import deepcopy
load_dotenv()

True

## load variables
- data folder
- cleaned_df - for only halfmoons
- json template
- 

In [48]:
data_folder = os.path.join(os.getcwd(),'data')
cleaned_csv = os.path.join(data_folder, 'final_range_data.csv')
dff = pd.read_csv(cleaned_csv)
cleaned_df = dff[dff['halfmoons-halfmoon_details-halfmoon_type'].notnull()]
halfmoon_df = pd.read_csv(os.path.join(data_folder, 'brics_csvs', 'cleaned_trees_grasses_halfmoons.csv'))

TEMPLATE_FILE = os.path.join(data_folder,"halfmoon_json_template.json")

with open(TEMPLATE_FILE, "r") as f:
    template = json.load(f)



## unique names from dataset

In [55]:
#sorted(list(dff['basic_info-name'].unique()))
##empty name and key dictionary
names_dict = {}
for idx, row in dff.iterrows():
    #ic({row["KEY"], row["basic_info-name"]})
    names_dict[row["KEY"]] = row["basic_info-name"]
#ic(names_dict)
name_uname_dict = {
    "Zakariye haji ali": "abdullahi.hassan@mardo.org", 
    "Issack Idow Ibrahim": "abdullahi.hassan@mardo.org",
    "Abdullahi Hassan Adan": "abdullahi.hassan@mardo.org",
    "Abdikheir Mohamed": "mukhtar.yusuf23",
    "Abdikheir Mohamed Ali": "mukhtar.yusuf23",
    "Abdikheir Mohamed Dahir": "mukhtar.yusuf23",
    "Ali": "mukhtar.yusuf23",
    "Ali Adan": "mukhtar.yusuf23",
    "Ali Aden": "mukhtar.yusuf23",
    "Fathi": "mukhtar.yusuf23", 
    "Fathi Abdirashid": "mukhtar.yusuf23",
    "Ahmed Omar Hassan": "AhmedOmar",
    "Maria Abdirahman Ali": "AhmedOmar",
    "ABSHIR ABDULLAH ALI": "Ahil",
    "Ahmed Ibrahim": "JubaFoundation",
    "Ahmed Ibrahim Mohamed": "JubaFoundation",
    "Abdiaziz Adan Hassa": "guudow",
    "Abdiaziz Adan Hassan": "guudow",
    "Abdi Hassan Adan": "AbdiHassan",
    "Abdullahi": "sharif1234",
    "Abdirashid Sheikh Mohamed": "NRMO",
    "Abdullahi Sharif": "sharif1234",
    "Abdullahi Sharif LLG": "sharif1234",
    "Abdullahi Sharif Noor LLG": "sharif1234",
    "Abdijalil": "Csxaashi",
    "Abdijalil said": "Csxaashi",
    "Abdikarim Mohamed Ismail": "samandoulgou",
    "Abdikarin Mohamed Ismail": "samandoulgou",
    "Hassan Omar Khadhib": "Khadhib",
    "Hassan Omar Khadhibr": "Khadhib",
    "Jamal Ali": "JamalAli",
    "Jamal Ali Tagal": "JamalAli",
    "Mohamud Abdullahi Ali": "Mohamud",
    "Moulid Abdullahi Abdi": "Moulidwadani",
    "Muktar Mohamednoor Rage": "Mrage1997",
    "Osman Ebey Omar": "iibey",
    "Yusuf Mahi munye": "Mcnbdn617660",
    "abdullahi Sharif LLG": "sharif1234",
    "osman Ebey Omar": "iibey",
}

## match and replace names with usernames
for k, v in names_dict.items():
    for key, value in name_uname_dict.items():
        if v == key:
            names_dict[k] = value
#ic(names_dict)

##enumerator dictionary
enumerator = {
        "id": 3726,
        "first_name": "First",
        "last_name": "Last",
        "gender": "Male",
        "age_category": "18 - 35",
        "phone_number": "0e15704046",
        "email": "test_mail@gmail.com",
        "country": "test",
        "organization": "Test",
        "username": "test",
        "type": "ENUMERATOR"
    }

enums = []
#create enumerator jsons
for key, value in names_dict.items():
    e = deepcopy(enumerator)
    e['username'] = value
    e['key'] = key
    enums.append(e)

ic(enums)

ic| enums: [{'age_category': '18 - 35',
             'country': 'test',
             'email': 'test_mail@gmail.com',
             'first_name': 'First',
             'gender': 'Male',
             'id': 3726,
             'key': 'uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47',
             'last_name': 'Last',
             'organization': 'Test',
             'phone_number': '0e15704046',
             'type': 'ENUMERATOR',
             'username': 'abdullahi.hassan@mardo.org'},
            {'age_category': '18 - 35',
             'country': 'test',
             'email': 'test_mail@gmail.com',
             'first_name': 'First',
             'gender': 'Male',
             'id': 3726,
             'key': 'uuid:b5ab198a-5c8b-4e40-90c4-f0efa1e969ff',
             'last_name': 'Last',
             'organization': 'Test',
             'phone_number': '0e15704046',
             'type': 'ENUMERATOR',
             'username': 'sharif1234'},
            {'age_category': '18 - 35',
             'coun

[{'id': 3726,
  'first_name': 'First',
  'last_name': 'Last',
  'gender': 'Male',
  'age_category': '18 - 35',
  'phone_number': '0e15704046',
  'email': 'test_mail@gmail.com',
  'country': 'test',
  'organization': 'Test',
  'username': 'abdullahi.hassan@mardo.org',
  'type': 'ENUMERATOR',
  'key': 'uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47'},
 {'id': 3726,
  'first_name': 'First',
  'last_name': 'Last',
  'gender': 'Male',
  'age_category': '18 - 35',
  'phone_number': '0e15704046',
  'email': 'test_mail@gmail.com',
  'country': 'test',
  'organization': 'Test',
  'username': 'sharif1234',
  'type': 'ENUMERATOR',
  'key': 'uuid:b5ab198a-5c8b-4e40-90c4-f0efa1e969ff'},
 {'id': 3726,
  'first_name': 'First',
  'last_name': 'Last',
  'gender': 'Male',
  'age_category': '18 - 35',
  'phone_number': '0e15704046',
  'email': 'test_mail@gmail.com',
  'country': 'test',
  'organization': 'Test',
  'username': 'sharif1234',
  'type': 'ENUMERATOR',
  'key': 'uuid:93a4de6b-e4d4-42ad-af3d-868aae8

### get names from db
the cell has two queries:
- get all somalia users
- get user details for json population

In [21]:
# create a db engine
engine = create_engine(os.getenv('DATABASE_URL1'))

#query 
query = """select id, username, first_name, last_name, email, country from respi_regreeningusers where country in ('Somalia', 'somalia') order by first_name asc;"""
query2 = """select id, first_name, last_name, gender, age_category, phone_number, email, country, organization, username from respi_regreeningusers where id in(3726, 4257,4548, 2495, 3228, 2226, 4243, 5113, 457, 1839, 4299, 3725, 3323, 1894, 4300, 4310, 5360, 5364, 5368);"""
#execute the query and fetch the results
conn = engine.connect()
df = pd.read_sql_query(query, conn)
df2 = pd.read_sql_query(query2, conn)

#close the connection
conn.close()
engine.dispose()

df2

,id,first_name,last_name,gender,age_category,phone_number,email,country,organization,username
0,457,Abdi,Adan,Male,18 – 35,0615049284,abdi.shekh07@gmail.com,Somalia,MARDO,AbdiHassan
1,1839,Abdullahi Noor,Ali,Male,18 – 35,252612405084,deerow248@gmail.com,Somalia,LIFELINE GEDO,sharif1234
2,1894,Abdijalil,Toreye,Male,36 – 59,252661731399,abdijalil.xaashi@kaalo.org,Somalia,Kaalo,Csxaashi
3,2226,Mohamud,Abdullahi,Male,18 – 35,252613134883,mohamudaabdullahi@gmail.com,Somalia,NRC,Mohamud
4,2495,Hassan Omar,Khadhib,Male,,0615807566,khadhiib10@gmail.com,Somalia,Norwegian Refugee Council,Khadhib
5,3228,Jamaal,Tagal,Male,18 – 35,252616719022,suldaanyare1265@gmail.com,Somalia,ACF,JamalAli
6,3323,Abdikheir,Mahamed,Male,18 – 35,0619997802,abdikheir.m@lifelineorganization.org,Somalia,Lifelinegedo,abdikheirMohamed
7,3725,Yusuf,Mahi,Male,18 – 35,252615702180,yusuf.munye@hijra.or.ke,Somalia,Hijra,Mcnbdn617660
8,3726,Abdiaziz Adan,Hassan,Male,18 – 35,0615704046,abdulazizadan02@gmail.com,Somalia,GREDO,guudow
9,4243,Moulidabdullahi,Abdi,Male,36 – 64,0615446004,moulidwadani@gmail.com,Somalia,SSWC,Moulidwadani


## enumerator
get user details from db for json population

In [22]:
for idx, i in enumerate(df2.to_dict(orient='records'), start=1):
    i['type'] = 'ENUMERATOR'
    ic(i)

ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'abdi.shekh07@gmail.com',
        'first_name': 'Abdi',
        'gender': 'Male',
        'id': 457,
        'last_name': 'Adan',
        'organization': 'MARDO',
        'phone_number': '0615049284',
        'type': 'ENUMERATOR',
        'username': 'AbdiHassan'}
ic| i: {'age_category': '18 – 35',
        'country': 'Somalia',
        'email': 'deerow248@gmail.com',
        'first_name': 'Abdullahi Noor',
        'gender': 'Male',
        'id': 1839,
        'last_name': 'Ali',
        'organization': 'LIFELINE GEDO',
        'phone_number': '252612405084',
        'type': 'ENUMERATOR',
        'username': 'sharif1234'}
ic| i: {'age_category': '36 – 59',
        'country': 'Somalia',
        'email': 'abdijalil.xaashi@kaalo.org',
        'first_name': 'Abdijalil',
        'gender': 'Male',
        'id': 1894,
        'last_name': 'Toreye',
        'organization': 'Kaalo',
        'phone_number': '2526617

## farmingEntity
- read column values from cleaned df and append to json keys

In [23]:
farming_entity_list = []
for idx, i in enumerate(cleaned_df.to_dict(orient='records'), start=1):
    farming_entity = {}
    #ic(i['basic_info-organisation_name'])
    farming_entity['id'] = idx
    farming_entity['first_name'] = i['basic_info-organisation_name']
    farming_entity['middle_name'] = i['basic_info-organisation_name']
    farming_entity['last_name'] = i['basic_info-organisation_name']
    farming_entity['organization'] = 'INSTITUTION'
    farming_entity['key'] = i['KEY']
    
    farming_entity_list.append(
        {
            "farmingEntity": farming_entity, 
            'key': i['KEY']
        }
    )
    ic(farming_entity_list)
    

ic| farming_entity_list: [{'farmingEntity': {'first_name': 'acf',
                                             'id': 1,
                                             'key': 'uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47',
                                             'last_name': 'acf',
                                             'middle_name': 'acf',
                                             'organization': 'INSTITUTION'},
                           'key': 'uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47'}]
ic| farming_entity_list: [{'farmingEntity': {'first_name': 'acf',
                                             'id': 1,
                                             'key': 'uuid:f8a33b7a-e769-41f7-b58e-422d1cc80f47',
                                             'last_name': 'acf',
                                             'middle_name': 'acf',
                                             'organization': 'INSTITUTION'},
                           'key': 'uuid:f8a33b7a-e769-41f7-b58e-422d

## plot-mcm_details
- read df to dictionary, for matching keys, extract df values
### plot-points
- parse the geoshape column from cleaned df
- split and extract the values from each i
- append to json keys

## crops
- get matching rows based on key(id)
- for these rows if crop has crops, get its values and 'other' if any
- clean 'and' from the crop types

In [24]:
import uuid
import copy
import re

def parse_geoshape(geoshape_str):
    #parse a geoshape string into a list of point dictionaries
    points = []
    parts = geoshape_str.split(';')

    for part in parts:
        part = part.strip()
        if not part:
            continue

        lat, lon, alt, acc = map(float, part.split(' '))

        points.append({
            "longitude": lon,
            "latitude": lat,
            "altitude": alt,
            "accuracy": acc
        })

    return points


plot_list = []
point_id = 1

records = cleaned_df.to_dict(orient='records')

count = 0
#iterate over every i to capture plot details
for idx, i in enumerate(records):
    plot_id = idx + 1
    count += 1

    # ---- BUILD PLOT DETAILS ----
    template_cp = copy.deepcopy(template[0])
    plot_details = template_cp['plot']

    plot_details['id'] = plot_id
    plot_details['key'] = i['KEY']
    plot_details['name'] = uuid.uuid4().hex
    plot_details['calculated_size'] = i['area_ha']
    plot_details['photo_url'] = i['plot_details-photo']
    plot_details['farmingEntityId'] = farming_entity['id']
    plot_details['livestock_allowed'] = False if i['land_use_selection-land_use'] == 'agriculture' else True
    plot_details['conservation_area'] = False

    #get only child row for this key
    matching_child_rows = halfmoon_df[halfmoon_df['PARENT_KEY'] == i['KEY']]
    
    
    """----------------------------------CROP STATUS ----------------------------------"""
    #iterate over matching child is to build crop status
    for x in matching_child_rows.to_dict(orient='records'):
        #plot_details['has_crops'] = True if x['crops_present'] == 'yes' else False
        #val = x['crops_in_field']
        if pd.isna(x['crops_in_field']):
            plot_details['has_crops'] = False
        else:
            plot_details['has_crops'] = True

        #get a list of all crops in field
        if pd.isna(x['crops_in_field']):
            crop_list = []
        else:
            crop_list = str(x['crops_in_field']).split(' ')

        #check that 'other' crops exist
        if 'other' in crop_list:
            other_crops_value = re.split(r'[,\s]+', x['other_crops'].strip())
            
            #get rid of 'and' if exists
            if 'and' in other_crops_value:
                other_crops_value.remove('and')

            #only add if not empty
            if other_crops_value:
                crop_list.remove('other')
                crop_list.extend(other_crops_value)
        
            #crop_list.remove('and')

        plot_details['crops'] = crop_list            

        
    """--------------------------------GEOSHAPE PARSING ----------------------------------"""
    geoshape_str = i['plot_details-polygon']
    parsed_points = parse_geoshape(geoshape_str)

    points_list = []
    for p in parsed_points:
        points_list.append({
            "id": point_id,
            "plotId": plot_id,
            "longitude": p["longitude"],
            "latitude": p["latitude"],
            "altitude": p["altitude"],
            "accuracy": p["accuracy"]
        })
        point_id += 1


    """--------------------------------ADMINISTRATIVE DETAILS ----------------------------------"""
    administrative = plot_details['subCounty']
    administrative['subcounty_name'] = i['geography-district_name']
    administrative['county']['county_name'] = i['geography-region_name']


    #update plot_details with number of points and administrative info
    plot_details['points'] = points_list
    plot_details['subCounty'] = administrative
    
    # del plot_details['points']
    # del plot_details['subCounty']
    general_plot = {
        "plot":plot_details, 
        'key':i['KEY']
    }
    #ic(i['KEY'])
    plot_list.append(
        general_plot
    )


In [25]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(plot_list, f, indent=4)

str(halfmoon_df['crops_in_field'][20]).split(' ')

['maize', 'sorghum', 'beans', 'tomato']

## project

In [26]:
projects_list = []

for row in cleaned_df.to_dict(orient='records'):
    name = row['land_use_selection-project_name']

    if name == 'terra':
        name = name.upper()
        id = 131
    elif name == 'brcis3':
        name = 'BRCiS III'
        id = 93

    projects_list.append({
        "project": {
            "id": id,
            "project_name": name
        },
        "key": row['KEY']
    })


## extract repeat csv records for cleaned data
1. get the 'KEY'(works like key and foreign key in the db) values from the main.csv to a list
2. For every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list
3. save the files

In [27]:
#step 1: get the 'KEY' values from cleaned_df to a list
keys = cleaned_df['KEY'].tolist()

#step 2.1: get the other csv filenames
dir = os.path.join(os.getcwd(),'data','brics_csvs')
main_csv = 'Rangeland_Somalia_BRCiSIII.csv'
#halfmoon_csv = 'Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv'
filenames = [file for file in os.listdir(dir) if file.endswith('.csv') and file != main_csv]

#step2.2 : for every df(repeat_csvs) get the row values where the 'PARENT_KEY' `.isin()` keys list

for name in filenames:
    #read the repeat csv
    doi = pd.read_csv(os.path.join(dir,name))

    #construct its new name
    new_filename = f'cleaned_{name}'

    #gey the filtered row values for mtching keys
    filtered_doi = doi[doi['PARENT_KEY'].isin(keys)]

    #step3: save the csvs
    filtered_doi.to_csv(os.path.join(dir,'outputs',new_filename), index=False)

    print(f'💯Created: {new_filename} 🎯with {len(filtered_doi)} records')


💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gully.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_contour.csv 🎯with 11 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_swales.csv 🎯with 62 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_keyline.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_gabions.csv 🎯with 6 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_terracing.csv 🎯with 0 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_bunds.csv 🎯with 2 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_halfmoons.csv 🎯with 212 records
💯Created: cleaned_Rangeland_Somalia_BRCiSIII-trees_grasses_basins.csv 🎯with 0 records
💯Created: cleaned_cleaned_trees_grasses_halfmoons.csv 🎯with 212 records


# Microcatchments
- loop through the the records and extract microcatchment data.

In [28]:
microcatchment_list = []
for idx, c in enumerate(records):

    """---------------------------------------MICROCATCHMENT DETAILS---------------------------"""
  
    mcm_details = {
        'key':c['KEY'],
        "id": idx + 1,
        "microcatchment_type": "Halfmoons",
        "total_microcatchments": c['halfmoons-halfmoon_details-total_halfmoons'],
        "length": c['halfmoons-halfmoon_details-length_decimal'],
        "width": c['halfmoons-halfmoon_details-width_decimal'],
        "depth": c['halfmoons-halfmoon_details-depth_decimal'],
        "vertical_spacing": c['halfmoons-halfmoon_details_2-vertical_space'],
        "horizontal_spacing": c['halfmoons-halfmoon_details_2-horizontal_space'],
        "established_date": datetime.strptime(c['halfmoons-halfmoon_details-date'], "%d/%m/%Y").strftime("%Y-%m"),
        #"reseeded": True if ,
        #"quantity_seeds_sown": 2.0,
        #"sow_unit": "KG",
        "seed_sources": [],
        "local_seed_bank_name": "",
        "other_seed_sources": "",
        "who_manages_microcatchment": ["MALE","FEMALE"] if c['halfmoons-halfmoon_details_2-manages'] == 'both' else ['MALE'],
        "other_who_manages": "",
        "management_practices": [
            "PLANTING_ANNUAL_PERENNIAL_CROPS"
        ],
        "other_managements": [],
        "usages": [],
        "other_usages": "",
        "rangeland_entry_id": 7,
        "completed": True,
    }
    
    if c['halfmoons-halfmoon_details_2-youth_manages'] == 'yes':
        mcm_details['who_manages_microcatchment'].append('YOUTH')

    '''--------------------------------------MANAGEMENT_PRACTICES--------------------------------'''
    #get only child rows for this record("KEY")    
    matching_mgmt_child_rows = halfmoon_df[halfmoon_df['PARENT_KEY'] == c['KEY']]
    ##check for matches
    # children = matching_mgmt_child_rows['PARENT_KEY'].tolist()
    # print(f"parent: ",c['KEY'], "child", len(children), children)

    practices = []
    child_records = matching_mgmt_child_rows.to_dict(orient='records')

    for  y in child_records:
        if y['additional_management'] == 'yes':
            vals = str(y['management_practices']).split(' ')
            practices.extend(vals)

    # make list a set - this removes duplicates/
    # the management practices in odk are halfmoon based,
    # in regreen app schema, they are microcatchment based
    # therefore we will be using unique management values only/
    set_practices = set(practices)
    #make all characters upper case(RA app schema requirement)
    ls_practices = [a.upper() for a in list(set_practices)]
    
    #dictionary to map odk values to RA app schema values
    mapping_dict = {
        "STICKS_BRANCH":"STICKS_PROTECTION",
        "DEBRIS_REMOVAL":"DEBRIS_REMOVAL",
        "INVASIVE_SPECIES":"INVASIVE_CONTROL",
        "WEEDING":"WEEDING",
        "WATERING":"WATERING",
        "PLANTING_SEEDLING":"PLANTING_ANNUAL_PERENNIAL_CROPS"
    }

    #map the odk values to accepted ra values
    normalised_practices = [v for k,v in mapping_dict.items() if k in ls_practices]
    mcm_details['management_practices'] = normalised_practices
    
    #for values not in ra, append them to other managements
    other_managements = [x for x in ls_practices if x not in mapping_dict.keys()]
    mcm_details['other_managements'] = other_managements

    """----------------------------------------seeded??----------------------------------------------"""

    # initialize aggregated values
    total_seeds = 0.0
    any_reseeded = False

    for k in child_records:

        # update global reseeded flag if this row is reseeded
        if  k['halfmoon_trees-planted_choice1'] in ['both', 'grasses']:
            any_reseeded = True

        # add seed quantity if numeric
        qty = k['halfmoon_grasses-kg_grass_seeds']
        if isinstance(qty, (int, float)) and not pd.isna(qty):
            total_seeds += qty

    # final aggregated dictionary
    seed_info = {
        "reseeded": any_reseeded,
        "quantity_seeds_sown": total_seeds if total_seeds > 0 else 0,
        "sow_unit": "KG" if any_reseeded else ""
    }

    #print(seed_info)

    # attach to mcm_details
    mcm_details.update(seed_info)




    #     seeded.extend(reseeded)

    """----------------------------------------CURRENT_STATUS-----------------------------------------"""

    cureent_status_list = []
    count = 0
    all_grasses = []
    all_trees = []

    for i, j in enumerate(child_records):
        #mapping dictionary for herbacious cover
        hb_cover_mapping = {
            1: "LESS_FOUR",
            2: "FOUR_FIFTEEN",
            3: "FIFTEEN_FOURTY",
            4: "FOURTY_SIXTFIVE",
            5: "SIXTHFIVE_ABOVE"
        }

        current_status = {
            'key':j['KEY'],
            "id": i + 1,
            "percentage_herbaceous_cover_in": hb_cover_mapping[j['herbaceous_cover']],
            "percentage_herbaceous_cover_between": hb_cover_mapping[j['herbaceous_cover_between']],
            "intact": True if j['halfmoon_intact'] == 'yes' else False,
            "protected": False,
            "protection_type": [],
            "other_protection_type": "",
            "visible_erosion": True if j['visible_erosion'] == 'yes' else False,
            "erosion_type_inside": [],
            "other_erosion_type_inside": "",
            "erosion_type_outside": [],
            "other_erosion_type_outside": "",
            "longitude": j['gps-Longitude'],
            "latitude": j['gps-Latitude'],
            "altitude": j['gps-Altitude'],
            "accuracy": j['gps-Accuracy'], 
            "photo_url": j['photo'],
            "comments": f"labor {c['halfmoons-halfmoon_details_2-paid_unpaid_labor']} state {c['geography-state_name'].replace("â€¦", "").replace(".", "").replace("..", "")} community {c['geography-community_name']} type {c['halfmoons-halfmoon_details_2-type_halfmoons']}",
            "microcatchment_id": 7,
            "grassCurrentStatus": []
        }

        comments_parts = []
        """----------------------------grass establishment---------------------------------------"""
        #initiate a list for grasses in one halfmoon
        grass_status_list = []

        if j['halfmoon_trees-planted_choice1'] in  ['grasses', 'both']:
            comments_parts.append(f"grassSpecies {j['halfmoon_grasses-num_grass_species']}")

            grass_species_list = str(j['halfmoon_grasses-halfmoon_grass_species']).split()
            #default
            scientific_name = ""
            local_name = ""

            for idx2 ,raw in enumerate(grass_species_list):
                #case 1: others
                if raw.strip() == 'other':
                    other_raw = str(j['halfmoon_grasses-other_halfmoon_grass'])

                    #split by comma & 'and'
                    names = re.split(r",| and ", other_raw, flags=re.IGNORECASE)

                    #loop through local names
                    for idx, name in enumerate(names):
                        cleaned = name.strip()
                        if cleaned != 'nan':
                            grass_current_status_1 = ({
                                "id": idx + 1,
                                "local_name": cleaned,
                                "scientific_name": "",
                                "microcatchment_status_id": 5
                            })
                        grass_status_list.append(grass_current_status_1)

                    continue
                
                #case2: ends with other and also has nan in other colun
                elif raw.strip().endswith('other'):
                    #use the 'other' column
                    local_name = str(j['halfmoon_grasses-other_halfmoon_grass']).strip()
                #case 3: split local name from scientific name
                else:
                    #split scientific and local names
                    if "(" in raw and ")" in raw:
                        #scientific name cleaning
                        parts = raw.split("(")[0].strip().replace("_", " ").split()
                        if len(parts) == 2:
                            scientific_name = f"{parts[0].capitalize()} {parts[1].lower()}"
                        else:
                            scientific_name = raw.split("(")[0].strip().replace("_", " ")

                        #local name cleaning
                        paarts = raw.split("(")[1].split(")")[0].strip().replace("_", " ").split()
                        if len(paarts) == 2:
                            local_name = f"{paarts[0].capitalize()} {paarts[1].lower()}"
                        else:
                            local_name = raw.split("(")[1].split(")")[0].strip().replace("_", " ").replace(",", " ")#[1].lower()
                    else:
                        scientific_name = raw.strip()

                if local_name != 'nan':
                    grass_current_status = {
                        "id": idx2 + 1,
                        "local_name": local_name,
                        "scientific_name": scientific_name,
                        "microcatchment_status_id": 5
                    }
            
                #append the current status for ths record to the empty list
                grass_status_list.append(grass_current_status)
        #add every status to the to serve as establishment        
        all_grasses.extend(grass_status_list)

        #set the grass establishments to current status
        current_status['grassCurrentStatus'] = grass_status_list
        #print(f"parent:{j['PARENT_KEY']}, child:{j['KEY']}")


        """---------------------------------tree establishment--------------------------------------"""
        tree_status_list = []
        if j['halfmoon_trees-planted_choice1'] in  ['trees', 'both']:
            comments_parts.append(f"treeSpecies {j['halfmoon_trees-num_tree_species']}")
            comments_parts.append(f"plantedTrees {j['halfmoon_trees-num_trees_planted']}")
            comments_parts.append(f"survivedTrees {j['halfmoon_trees-num_tree_survived']}")

            trees_species_list = j['halfmoon_trees-halfmoon_trees_species'].split()
            #ic(c['KEY'])
            #ic(trees_species_list)
            
            for idx3, raw in enumerate(trees_species_list):

                #default
                scientific_name = ""
                local_name = ""
                 
                #case1: check or other in the tree species list
                if raw == 'other':
                    other_val = str(j['halfmoon_trees-other_halfmoon_trees']).strip()

                    #split by comma & 'and'
                    names = re.split(r",| and ", other_val, flags=re.IGNORECASE)

                    for idxx, val in enumerate(names):
                        local_name = val.strip()
                        #handle nan cases
                        if local_name != 'nan' and local_name != "":
                            tree_current_status = {
                                "id": idxx + 1,
                                "local_name": local_name.capitalize(),
                                "scientific_name": scientific_name,
                                "microcatchment_id": 8
                            }
                            #ic(tree_current_status)
                            tree_status_list.append(tree_current_status)
                        

                #case 2: scientific + local name extraction
                if "(" in raw and ")" in raw:
                    #scientific name cleaning
                    parts = raw.split("(")[0].strip().replace("_", " ").split()
                    
                    if len(parts) == 2:
                        scientific_name = f"{parts[0].capitalize()} {parts[1].lower()}"
                        #ic(scientific_name)

                    #local name cleaning
                    paarts = raw.split("(")[1].split(")")[0].strip().replace("_", " ").replace(",", " ").split()
                    #ic(paarts)
                    if len(paarts) == 2:
                        local_name = f"{paarts[0].capitalize()} {paarts[1].lower()}"
                        #ic(local_name)
                    else:
                        local_name = paarts[0].capitalize()
                        #ic(local_name)

                #case2: handle other trees 
                #elif raw.endswith('other') or 'other' in raw:
                    #ic(raw)

                tree_current_status = {
                    "id": idx3 + 1,
                    "local_name": local_name,
                    "scientific_name": scientific_name,
                    "microcatchment_id": 8
                }
                #ic(tree_current_status)
                tree_status_list.append(tree_current_status)
        
        #buld final comment string
        final_coment = " ".join(comments_parts)
        current_status["comments"] = current_status["comments"] + " " + final_coment

        # add every tree status to establishment list
        all_trees.extend(tree_status_list)
        #ic(all_trees)
        #append tree status list to key in status dictionary
        current_status['treeCurrentStatus'] = tree_status_list
        
        #a list of all current status fields from tree to grasses
        cureent_status_list.append(current_status)
            
    

    mcm_details['currentStatus'] = cureent_status_list
    mcm_details['grassEstablishment'] = all_grasses
    mcm_details['treeEstablishment'] = all_trees
    microcatchment_info = {
        "microcatchment":mcm_details, 
        'key':j['PARENT_KEY']
    }
    #ic(j['PARENT_KEY'])
    microcatchment_list.append(microcatchment_info)
    
    

In [29]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(microcatchment_list, f, indent=4)


## merge components to one json


In [31]:
# component lists
lists = [farming_entity_list, microcatchment_list, plot_list, projects_list]

# The output dictionary
merged = {}

for lst in lists:
    ic(len(lst))
    for item in lst:
        k = item['key']

        #check if already updated
        if k not in merged:
            merged[k] = {}
            
        #merge the item into the existing dictionary
        merged[k].update(item)

final = list(merged.values())
for i in final:
    i["boma"] = []
    i["cgrazing"] = []
    i["econtrol"]= []
    i["seedbank"] = []
    i["seeding"] = []
    i["waterpoint"] = []
    i["completed"] = True
    i["rangeModuleId"] = 1
    i["currentStep"]= "DONE"
    i["moduleId"] = 6
    i["is_revisit"] = False
    #get the date_collected
    for j in records:
        if i["key"] == j["KEY"]:
            i["date_collected"] = datetime.strptime(j['basic_info-date'], "%d/%m/%Y").strftime("%Y-%m-%d")
    #match project id
    for x in projects_list:
        if i['key'] == x['key']:
            i['projectId'] = x['project']['id']
    #match plot id
    for k in plot_list:
        if i['key'] == k['key']:
            i['plotId'] = k['plot']['id']
    #match farming_entity_id
    for h in farming_entity_list:
        if i['key'] == h['key']:
            i['farmingEntityId'] = h['farmingEntity']['id']



ic(len(final))


ic| len(lst): 55
ic| len(lst): 55
ic| len(lst): 55
ic| len(lst): 55
ic| len(final): 55


55

In [32]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(final, f, indent=4)

## clean up final json

In [ ]:
#clean up keys
def remove_keys(obj, key="key"):
    if isinstance(obj, dict):
        obj.pop(key, None)
        for v in obj.values():
            remove_keys(v, key)
    elif isinstance(obj, list):
        for v in obj:
            remove_keys(v, key)
    return obj

finaal = [x for x in final if x['key'] == 'uuid:b4a2d545-66fe-4d23-825b-41025149b460']
finaal = [remove_keys(i) for i in final]

#add id
for idx, c in enumerate(finaal):
    c['id'] = idx + 1


In [ ]:
with open(os.path.join(data_folder, 'to_delete.json'), 'w') as f:
    json.dump(finaal, f, indent=4)